### Get all the info box for all movies

In [1]:
from bs4 import BeautifulSoup as bs
import requests

### Load the webpage

In [2]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

#Convert to a beautifulsoup object
soup = bs(r.content, 'html.parser')

# content = soup.prettify()
# content

In [4]:
#soup.find_all(class_='wikitable sortable jquery-tablesorter')은 []을 리턴한다. 
#즉 find로 찾아지지 않으면 조금더 정확하게 찾을수 있는 CSS선택자로 찾자
#그래서 .select로 진행 .class1.class2 라는 CSS선택자 형태로 찾아 보자

# 1. CSS선택자로 테이블 형태를 찾는다
# movies = soup.select('.wikitable.sortable') # 이렇게 하면 찾아 진다. 

# 2. 테이블 구성을 보니 <tbody> -> <tr> -> <td> -> <i> 태그가 링크를 가지고 있다
movies = soup.select('.wikitable.sortable i') # 이렇게하면 i를 포함하고있는 '.wikitable.sortable i' 테이블을 리턴
# movies[0:10]
#print(movies[0].a['href']) #movies에서 [0]번째 인덱스 위치의 <a>태그가 가진 'href' 출력
#print(movies[0].a['title']) #movies에서 [0]번째 인덱스 위치의 <a>태그가 가진 'title' 출력

In [5]:
def get_content_value(row_data):
#     print(row_data)
    
    if row_data.find('li'):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all('li')]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ")

def get_info_box(url):
    
    r = requests.get(url)

    #Convert to a beautifulsoup object
    soup = bs(r.content, 'html.parser')
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all('tr')
    
    movie_info = {}
    
    for idx, row in enumerate(info_rows):
        if idx == 0: # 타이틀의 인덱스는 = 0
            movie = row.find('th').get_text(" ", strip=True)
        elif idx == 1:
            continue
        else:
            content_key = row.find('th').get_text(" ", strip=True)
#             print(content_key)
            content_value = get_content_value(row.find('td')) #이 부분이 get_content_value를 탐
            movie_info[content_key] = content_value
        
    return movie_info

In [4]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

#Convert to a beautifulsoup object
soup = bs(r.content, 'html.parser')


# movies = soup.select('.wikitable.sortable i')으로는 아래의 Nonetype이 있으므로 조금 더 구체적으로 가져오자
movies = soup.select('.wikitable.sortable i a') #즉 <i>태그 중 <a>를 가진 것만 들고 옴
# print(len(movies)) #NoneType 제외하고 개수를 확인 510개로 나옴


base_path = "https://en.wikipedia.org/" #기본 url을 세팅 후


movie_info_list = [] #들고 오는 내용을 담을 리스트
for index, movie in enumerate(movies):
    #디버깅용
#     if index == 10:
#         break     
    try:
        relative_path = movie['href']
        full_path = base_path + relative_path #긁어오는 'href'를 더해서 full_path 만든 후 함수에 삽입
        title = movie['title']
#         relative_path = movie.a['href'] # <i>태그 중 <a>만 들고 오니 .a는 생략
#         title = movie.a['title']
        #위의 결과들을 print 해보면 relative_path가 가져오는 'href'가 없는 부분이 있다. 여기서 try/catch가 필요
#         print(relative_path)
#         print(title)
#         print()
        
        movie_info_list.append(get_info_box(full_path)) #여기에 들어갈 url 세팅 (base_path + full_path)
        
    except Exception as e: #즉 오류부분이면 여기를타고 아래를 실행 
        print(movie.get_text()) #이 부분에서 어떤 movie에서 오류가 나오는지 보인다
        #몇몇의 movie가 테이블의 Title에서 'href'를 들고오는게아닌 'Note'부분에서 들고 온다. 
        #몇몇의 movie가 테이블의 Title에서 'href'를 들고 오지만, 링크가 아는 형태이다.
        
        #최종코드에서 돌려보면 여전히 몇몇의 movie가 find/find_all/get_text가 안된다
        print(e)

Zorro the Avenger
'NoneType' object has no attribute 'find'
The Sign of Zorro
'NoneType' object has no attribute 'find'
True-Life Adventures
'NoneType' object has no attribute 'find_all'
The London Connection
'NoneType' object has no attribute 'find'
Spirited Away
'NoneType' object has no attribute 'get_text'
Howl's Moving Castle
'NoneType' object has no attribute 'get_text'
The Nightmare Before Christmas 3D
'NoneType' object has no attribute 'get_text'
The Secret of the Magic Gourd
'NoneType' object has no attribute 'get_text'
Ponyo
'NoneType' object has no attribute 'get_text'
Tales from Earthsea
'NoneType' object has no attribute 'get_text'
Anaganaga O Dheerudu
'NoneType' object has no attribute 'get_text'
The Secret World of Arrietty
'NoneType' object has no attribute 'get_text'
Tini: The Movie
'NoneType' object has no attribute 'get_text'
Born in China
'NoneType' object has no attribute 'get_text'
Sister Act 3
'NoneType' object has no attribute 'find'
Tower of Terror
'NoneType' ob

In [21]:
len(movie_info_list)

{'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release date': ['May 19, 1937 ( 1937-05-19 )'],
 'Running time': '41 minutes (74 minutes 1966 release)',
 'Country': 'United States',
 'Language': 'English',
 'Box office': '$45.472'}

In [ ]:
#movie_info_list[0]

### Save/Reload Movie Data

In [8]:
import json

#JSON 형태로 저장
def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        #json.dumps() = Python 객체를 JSON으로 변환
        #ensure_ascii - true면 모든 비 ASCII 문자가 출력 안됨, False면 문자 그대로 출력
        #indent - 인덱스폭 (문자 수) 지정
        json.dump(data, f, ensure_ascii=False, indent=2)

In [9]:
#JSON 형태로 로드
def load_data(title):
    with open(title, encoding='utf-8') as f:
        return json.load(f)

In [10]:
save_data("Disney_data.json", movie_info_list)